In [1]:
import warnings
#warnings.filterwarnings('ignore')

# General packages                                                                                                                                                                                                                                                                                                                                                                                                                        import pandas as pd 
import numpy as np
import pandas as pd
import requests
import time

# regular expressions
import re

# BeautifulSoup
from bs4 import BeautifulSoup

from urllib.parse import urljoin

# Selenium packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service as ChromeService 
from webdriver_manager.chrome import ChromeDriverManager 

In [2]:
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9'}
# Open tab in full screen (maximized)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")

In [3]:
# Instantiate Selenium Webdriver
driver = webdriver.Chrome(options = chrome_options)

# Get the main page (Top Restaurants in Mumbai on Zomato)
driver.get("https://www.zomato.com/hyderabad/restaurants")

SCROLL_PAUSE_TIME = 2

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
#driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
#time.sleep(100)
# Get page soure
source = driver.page_source

In [4]:
# Get title of  main window
main_window_title = driver.title
print(main_window_title)

Restaurants in Hyderabad - Zomato


In [5]:
# creating soup
soup = BeautifulSoup(source, "html.parser")
rest_divs = soup.findAll('div', class_='jumbo-tracker')

In [6]:
# creating all the list here according to data
urls = []
rest_name = []
ratings = []
price = []
cuisine = []
location = []

remove = ["\’","\'","&"]

In [7]:
for rest in rest_divs:  

    # name of the restaurant is stored in the h4 tags
    name_tag = rest.find("h4")    
    if name_tag!= None:
        rest_name.append(name_tag.text)
        #print(name_tag.text)
        rating_tag = rest.div.a.next_sibling.div.div.div.div.div.div.div.text
        price_tag = rest.div.a.next_sibling.p.next_sibling.text
        cuisine_tag = rest.div.a.next_sibling.p.text
        ratings.append(rating_tag)
        price.append(price_tag)
        cuisine.append(cuisine_tag)
        
        # links of the restaurants from <a> tags, getting href where links are stored
        link_tag = rest.find("a")
        if link_tag!=None:
            # since we don not get whole link https attached we need to join the link with base
            base = "https://www.zomato.com"  
            try:
                if 'href' in link_tag.attrs:
                    link = link_tag.get('href')
            except:
                pass             
            url = urljoin(base, link)
            urls.append(url)
            #getting location of the restaurant
            name_text = ''.join(x for x in name_tag.text if x.isalnum()).lower()
            link_li = link.split('/')
            loc = link_li[2].replace('-','').replace(name_text,'')
            b = re.findall(r'\d+', loc)
            if len(b)!=0:
                location.append(loc.replace(b[0],''))
            else:
                location.append(loc)
            #print(name_text.lower(),'====',loc)    
print(len(rest_name))
print(len(location))

1122
1122


In [8]:
zomato_df = pd.DataFrame({'links': urls, 'names': rest_name, 'ratings': ratings, 'price for one': price, 'cuisine': cuisine, 'location': location})

In [9]:
zomato_df.to_csv("Hyderabad_restaurants_Main.csv")
#  creating csv for information
#driver.close()